In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# New section

In [ ]:
# import the required libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import re
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize    
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
#from sklearn import cross_validation
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import pickle

import nltk
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#loading the data files
fake_data = pd.read_csv("/content/drive/MyDrive/Fake.csv")
true_data = pd.read_csv("/content/drive/MyDrive/True.csv")


In [ ]:
#adding the label col
fake_data["label"]="Fake"
true_data["label"]="True"

In [ ]:
true_data.dtypes

title      object
text       object
subject    object
date       object
label      object
dtype: object

In [ ]:
fake_data.dtypes

title      object
text       object
subject    object
date       object
label      object
dtype: object

In [ ]:
np.random.seed(10)
#fake_data= fake_data.sample(10900)
#true_data= true_data.sample(10000)

In [ ]:
#merging the fake and true files to prepare the train_data file
frames= [fake_data,true_data]
train_data = pd.concat(frames)

In [ ]:
train_data.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [ ]:
#dropping the date col
train_data.drop("date", axis=1, inplace=True)

In [ ]:
train_data=train_data.reset_index() #reseting index of the train_data
train_data=train_data.sample(frac = 1) #shuffling the data
train_data=train_data.sample(frac=1)
train_data.head()

,index,title,text,subject,label
38136,14655,Two Turkish soldiers killed in northern Iraq: ...,ISTANBUL (Reuters) - Two Turkish soldiers were...,worldnews,True
19900,19900,OUCH! WATCH: Sneaky Liberal Gets Electrocuted ...,"Punishment is not for revenge, but to lessen ...",left-news,Fake
44204,20723,EU tells Britain to protect data or delete the...,BRUSSELS (Reuters) - The European Union wants ...,worldnews,True
44272,20791,Factbox: Caribbean and Gulf oil companies begi...,(Reuters) - Gulf Coast and Caribbean energy in...,worldnews,True
23718,237,Special Counsel Mueller produces evidence that...,WASHINGTON (Reuters) - U.S. Special Counsel Ro...,politicsNews,True


In [ ]:
#encoding train_data.label as 0,1
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y = le.fit_transform(train_data.label)
y

array([1, 0, 1, ..., 0, 0, 1])

In [ ]:
# 0--> Fake
# 1--> True

In [ ]:
train_data['label']

38136    True
19900    Fake
44204    True
44272    True
23718    True
         ... 
9558     Fake
31221    True
2075     Fake
22975    Fake
28217    True
Name: label, Length: 44898, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, label_train, label_test = train_test_split(train_data['text'], y, test_size=0.2, random_state = 42)

print(len(X_train), len(X_test), len(X_train) + len(X_test))

35918 8980 44898


In [ ]:
#lemmatizing the text : only keeping the rrots of the words
#tokenizing the text : splitting the words into smaller units 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl=WordNetLemmatizer()
    def __call__(self,doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [ ]:
#vectorizing the words and making n-grams
from sklearn.feature_extraction.text import TfidfVectorizer

text_stopwords=stopwords.words()
vectorizer=TfidfVectorizer(analyzer='word',input='content',
                           lowercase=True,
                           token_pattern='(?u)\\b\\w\\w+\\b',
                           min_df=3,
                           ngram_range=(1,3),
                           tokenizer=LemmaTokenizer(),stop_words=text_stopwords)

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
x_train=vectorizer.fit_transform(X_train) # Preprocessing the train text data 
        
x_test=vectorizer.transform(X_test) # Preprocessing the test text data 


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", "'d", "'ll", "'m", "'re", "'s", "'ve", '.', '?', 'aldiz', 'avon', 'baizik', 'ban', 'bukatzeko', 'diesis', 'doe', 'dy', 'edota', 'ell', 'eze', 'ezpabere', 'ezpada', 'ezperen', 'f', 'fin', 'forum', 'fuss', 'gainera', 'gainerontzean', 'give', 'greeting', 'guztiz', 'hainbestez', 'horra', 'jinni', 'mus', "n't", 'onların', 'ordea', 'osterantzean', 'quantum', 'quine', 'reed', 'regard', 'serum', 'sha', 'sol', 'tote', 'wa', 'δ', 'δι', 'агар-чи', 'аз-баски', 'афташ', 'бале', 'баҳри', 'болои', 'валекин', 'вақте', 'вуҷуди', 'гар', 'гарчанде', 'даме', 'карда', 'кошки', 'куя', 'кӣ', 'магар', 'майлаш', 'модоме', 'нияти', 'онан', 'оре', 'рӯи', 'сар', 'тразе', 'хом', 'хуб', 'чаро', 'чи', 'чунон', 'ш', 'шарте', 'қадар', 'ҳай-ҳай', 'ҳамин', 'ҳатто', 'ҳо', 'ҳой-ҳой', 'ҳол', 'ҳолате', 'ӯим', 'באיזו', 'בו', '

In [ ]:
#implementing multinomial naive bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
model_lg = MultinomialNB()
#model training 
model_lg.fit(x_train, label_train)
# evaluate model
cv = KFold(n_splits=30, random_state=1, shuffle=True)
scores = cross_val_score(model_lg, x_train, label_train, scoring='f1', cv=cv, n_jobs=-1)
print(scores)

[0.98360656 0.98381295 0.9826087  0.98757249 0.98714653 0.9751773
 0.98606272 0.98774741 0.9867374  0.98263889 0.98418278 0.9846678
 0.98562975 0.97608503 0.98632479 0.98209719 0.98418278 0.98963731
 0.97980685 0.98714653 0.97931034 0.97913043 0.98426573 0.98791019
 0.98322148 0.98671391 0.98659966 0.98984772 0.98113208 0.98165939]


In [ ]:
# report performance
test_acacuracy=model_lg.score(x_test,label_test)
print(test_acacuracy)

0.9820712694877506


In [ ]:
#predicting on x_test
predictions2=model_lg.predict(x_test)

In [ ]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(predictions2,label_test))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      4553
           1       1.00      0.97      0.98      4427

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



In [ ]:
#TestCheck
input_text="Two days after his mother's death thrust him onto the throne, King Charles III was officially announced as Britain’s monarch Saturday, in a pomp-filled ceremony steeped in ancient tradition and political symbolism — and, for the first time, broadcast live. Charles, who spent seven decades as heir apparent, automatically became king when his mother, Queen Elizabeth II, died on Thursday, but the accession ceremony was a key constitutional and ceremonial step in introducing the new monarch to the country, a relic of a time before."

input_text=vectorizer.transform([input_text])

In [ ]:
model_lg.predict(input_text)

array([1])

In [ ]:
#creating the model
import joblib
joblib.dump(model_lg,'text.joblib')

['text.joblib']

In [ ]:
#loading the saved model
rf_model=joblib.load('text.joblib')

In [ ]:
#doing predictions
rf_model.predict(input_text)

array([1])